In [1]:
import sympy as sp
from IPython.display import display

In [30]:
f = sp.symbols('f', positive=True, real=True)
k = sp.symbols('kk', real=True)

xp0, xp1 = sp.symbols('x1[k](0), x1[k](1)', real=True)

x0 = xp0 / f
y0 = xp1 / f
r2 = x0 * x0 + y0 * y0

xx0 = x0
xx1 = y0
xx2 = 1 + k * r2
inv_norm = 1 / sp.sqrt(xx0 ** 2 + xx1 ** 2 + xx2 ** 2)

xx0 *= inv_norm
xx1 *= inv_norm
xx2 *= inv_norm

# x0 = sp.Function('x0')(xp0, f)
# x1 = sp.Function('x1')(xp1, f)
# r2 = sp.Function('r2')(xp0, xp1)
# xx0u = sp.Function('xx0u')(x0)
# xx1u = sp.Function('xx1u')(x1)
# xx2u = sp.Function('xx2u')(r2, k)
# xx0 = sp.Function('xx0')(xx0u, xx1u, xx2u)
# xx1 = sp.Function('xx1')( xx0u, xx1u, xx2u)
# xx2 = sp.Function('xx2')(xx0u, xx1u, xx2u)
# inv_norm = sp.Function('inv_norm')(xx0, xx1, xx2)


Jp = sp.zeros(3, 4)

Jp[0, 0] = (0 - xp0) * (-f * (xx0 * xx0 - 1) + 2 * xx0 * xx2 * k * (0 - xp0))
Jp[0, 0] += xx0 * (0 - xp1) * (-f * xx1 + 2 * xx2 * k * (0 - xp1))
Jp[0, 1] = f * (xx0 * xx0 - 1) - 2 * xx0 * xx2 * k * (0 - xp0)
Jp[0, 2] = xx0 * (f * xx1 - 2 * xx2 * k * (0 - xp1))
Jp[0, 3] = -xx0 * xx2 * r2
Jp[1, 0] = xx1 * (0 - xp0) * (-f * xx0 + 2 * xx2 * k * (0 - xp0))
Jp[1, 0] += (0 - xp1) * (-f * (xx1 * xx1 - 1) +
										2 * xx1 * xx2 * k * (0 - xp1))
Jp[1, 1] = xx1 * (f * xx0 - 2 * xx2 * k * (0 - xp0))
Jp[1, 2] = f * (xx1 * xx1 - 1) - 2 * xx1 * xx2 * k * (0 - xp1)
Jp[1, 3] = -xx1 * xx2 * r2
Jp[2, 0] = (0 - xp0) * (-f * xx0 * xx2 + 2 * k * (0 - xp0) * (xx1 * xx1 - 1))
Jp[2, 0] += (0 - xp1) * (-f * xx1 * xx2 + 2 * k * (0 - xp1) * (xx1 * xx1 - 1))
Jp[2, 1] = f * xx0 * xx2 - 2 * k * (0 - xp0) * (xx1 * xx1 - 1)
Jp[2, 2] = f * xx1 * xx2 - 2 * k * (0 - xp1) * (xx1 * xx1 - 1)
Jp[2, 3] = r2 * (1 - xx2 * xx2)
Jp[:, 0] /= f ** 3
Jp[:, 1] /= f ** 2
Jp[:, 2] /= f ** 2
Jp *= inv_norm

Jp = Jp[:, [0, 3]]

for i in range(3):
	for j in range(2):
		Jp[i, j] = Jp[i, j].simplify()

# JJp00 = sp.diff(xx0, f)
# print("Parsed Jp[0, 0]")
# print(Jp[0, 0])
# print("Sympy Jp[0, 0]")
# print(JJp00)
# print("Difference")
# print(sp.simplify(Jp[0, 0] - JJp00))

# print(Jp)
# display(Jp)

dJpdxp0 = sp.diff(Jp, xp0)
dJpdxp1 = sp.diff(Jp, xp1)

xxx0, xxx1, xxx2, r = sp.symbols('d1[k](0), d1[k](1), d1[k](2), r2')
dJpdxp0 = dJpdxp0.subs(r2, r).subs(xx0.subs(r2, r), xxx0).subs(xx1.subs(r2, r), xxx1).subs(xx2.subs(r2, r), xxx2)
dJpdxp1 = dJpdxp1.subs(r2, r).subs(xx0.subs(r2, r), xxx0).subs(xx1.subs(r2, r), xxx1).subs(xx2.subs(r2, r), xxx2)

In [38]:
from sympy.codegen.ast import CodeBlock, Assignment
from sympy.codegen.rewriting import optimize, optims_c99
from sympy.codegen.ast import real, float64

from sympy.codegen.rewriting import create_expand_pow_optimization
expand_opt = create_expand_pow_optimization(4)



a = []
for i in range(3):
	for j in range(2):
		sym = sp.Symbol(f'dp_e1({i}, {j})')
		# a.append(Assignment(sym, optimize(expand_opt(dJpdxp0[i, j]), optims_c99)))
		a.append(Assignment(sym, dJpdxp0[i, j]))
for i in range(3):
	for j in range(2):
		sym = sp.Symbol(f'dp_e2({i}, {j})')
		# a.append(Assignment(sym, optimize(expand_opt(dJpdxp1[i, j]), optims_c99)))
		a.append(Assignment(sym, dJpdxp1[i, j]))

code_block = CodeBlock(*a)

def optimize_code_block(code_block):
	optimized_assignments = []
	for assignment in code_block:
		name = assignment.lhs  # The left-hand side (symbol name)
		expr = assignment.rhs       # The right-hand side (expression)

		# Apply the optimization to the expression
		optimized_expr = expand_opt(expr)
		optimized_assignments.append(Assignment(name, optimized_expr))

	code_block = CodeBlock(*optimized_assignments)
	return code_block

code_block = optimize_code_block(code_block)

print("Non cse version")
print(sp.ccode(code_block, type_aliases={real: float64}).replace('pow', 'std::pow'))


print("cse version")
code_block = code_block.cse(symbols=sp.utilities.iterables.numbered_symbols(prefix='c'), optimizations='basic')
code_block = optimize_code_block(code_block)
print(sp.ccode(code_block, type_aliases={real: float64}).replace('pow', 'std::pow'))

Non cse version
dp_e1(0, 0) = x1[k](0)*std::pow(2*kk*(x1[k](0)*x1[k](0))*(f*f) + 2*kk*(x1[k](1)*x1[k](1))*(f*f) + 2*(kk*kk)*(x1[k](0)*x1[k](0))*(x1[k](1)*x1[k](1)) + (kk*kk)*(x1[k](0)*x1[k](0)*x1[k](0)*x1[k](0)) + (kk*kk)*(x1[k](1)*x1[k](1)*x1[k](1)*x1[k](1)) + (x1[k](0)*x1[k](0))*(f*f) + (x1[k](1)*x1[k](1))*(f*f) + f*f*f*f, -5.0/2.0)*(2*(kk*kk)*(x1[k](0)*x1[k](0))*(x1[k](1)*x1[k](1)) + (kk*kk)*(x1[k](0)*x1[k](0)*x1[k](0)*x1[k](0)) + (kk*kk)*(x1[k](1)*x1[k](1)*x1[k](1)*x1[k](1)) - (f*f*f*f))*(-6*kk*x1[k](0)*f*f - 3*x1[k](0)*f*f - 6*x1[k](0)*kk*kk*x1[k](1)*x1[k](1) - 6*kk*kk*x1[k](0)*x1[k](0)*x1[k](0)) + x1[k](0)*std::pow(2*kk*(x1[k](0)*x1[k](0))*(f*f) + 2*kk*(x1[k](1)*x1[k](1))*(f*f) + 2*(kk*kk)*(x1[k](0)*x1[k](0))*(x1[k](1)*x1[k](1)) + (kk*kk)*(x1[k](0)*x1[k](0)*x1[k](0)*x1[k](0)) + (kk*kk)*(x1[k](1)*x1[k](1)*x1[k](1)*x1[k](1)) + (x1[k](0)*x1[k](0))*(f*f) + (x1[k](1)*x1[k](1))*(f*f) + f*f*f*f, -3.0/2.0)*(4*x1[k](0)*(kk*kk)*(x1[k](1)*x1[k](1)) + 4*(kk*kk)*(x1[k](0)*x1[k](0)*x1[k](0))) 